# Fig. S18 - Sustained activity during simple sequences

In [ ]:
import json
import os
import pickle
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sylseq_paper.plotting import ucsf_sequential_color_palette as colors, default_plot_settings

In [ ]:
# Set up paths
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
img_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'imaging'))
mni_img_path = os.path.join(img_dir, 'MNI_{}_{}_brain_2D.png')

# Load bad channels
with open(os.path.join(data_dir, 'bad_channels.pkl'), 'rb') as f:
    bad_channels = pickle.load(f)

plot_subjects = ['EC217', 'EC219', 'EC223', 'EC237', 'EC240', 'EC241', 'EC253',
                 'EC254', 'EC260', 'EC263', 'EC276']

alignment_shorthand = {
    'target_presentation': 'target_pres',
    'delay'              : 'delay',
    'pre_exec'           : 'pre_exec',
    'speech'             : 'speech'
}
sig_thresh = 0.05

In [ ]:
# Load dataframe electrode information
with open(os.path.join(data_dir, 'fig1_ndf_sustained_se.pkl'), 'rb') as f:
    results = pickle.load(f)
    complex_sustained_se = results['sustained_se']

sig_above_baseline_df = pd.read_hdf(os.path.join(data_dir, 'suppfig_simple_sustained.h5'))

all_bad_se = []

for subject in bad_channels.keys():
    all_bad_se.extend(f'{subject}_{e}' for e in
                      np.concatenate(list(bad_channels[subject].values())))
    
# Remove bad channels
sig_above_baseline_df = sig_above_baseline_df.loc[~sig_above_baseline_df.subject_electrode.isin(all_bad_se)]

In [ ]:
simple_sustained_se = []
for se in sig_above_baseline_df.subject_electrode.unique():
    
    cur_df = sig_above_baseline_df.loc[sig_above_baseline_df.subject_electrode == se]
    pvals = np.array([cur_df[f'{k}_above_baseline_fdr_pvalues'].values[0] for k in alignment_shorthand.values()])
    is_sig = np.where(pvals < sig_thresh, True, False)

    if is_sig.all():
        simple_sustained_se.append(se)

simple_sustained_se = np.array(simple_sustained_se)

## Figure

In [ ]:
default_plot_settings(fontsize=11)

fig, axs = plt.subplots(2, 2, figsize=(7.5, 5), gridspec_kw=dict(height_ratios=[2, 1.25], wspace=0.05, hspace=0.1))

for r, view in enumerate(['lateral', 'medial']):
    for c, hemi in enumerate(['rh', 'lh']):
        
        ax = axs[r, c]
        
        img = plt.imread(mni_img_path.format(hemi, view))
        ax.imshow(img, alpha=0.75)
        
        simple_df = sig_above_baseline_df.loc[sig_above_baseline_df.subject_electrode.isin(simple_sustained_se)]
        cur_df = simple_df.loc[(simple_df.view == view) & (simple_df.hemisphere == hemi)]
        ax.scatter(cur_df.x, cur_df.y, color=colors[5], alpha=0.75, s=5, zorder=2, label='Simple\nsequences')
        
        complex_df = sig_above_baseline_df.loc[sig_above_baseline_df.subject_electrode.isin(complex_sustained_se)]
        cur_df = complex_df.loc[(complex_df.view == view) & (complex_df.hemisphere == hemi)]
        ax.scatter(cur_df.x, cur_df.y, facecolor='none', edgecolor=colors[1], alpha=0.75, s=7, zorder=1,
                  label='Complex\nsequences')
        
        ax.axis('off')
        
axs[1, 0].legend(frameon=False, bbox_to_anchor=(0.8, 1.3));